In [1]:
# 在softmax回归中，我们的输出与类别一样多

In [7]:
# 实现softmax相关函数，类。
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 初始化参数（此处均视为行向量）
num_inputs = 784 # 输入xi，d个
num_outputs = 10 # 输出类别数q个分类的概率oi，q个

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True) # d*q: o= xW+b
b = torch.zeros(num_outputs, requires_grad=True) # 1*q

def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了按行广播机制，X_exp:n*d, partition：n*1

# 定义模型o= XW+b
def net(X):
    # reshape:X格式化为n*d（n个行向量）, W:d*q（q为输出分类数）
    # W.shape[0] 是张量 W 的第一个维度的大小，通常表示行数或特征数。
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

# 定义交叉熵损失函数(适用于分类问题)
def cross_entropy(y_hat, y):
    '''返回每个分类真是概率的 -log值，等价−∑yilog(pi)，yi=(1, 0)(0, 1)：one-hot编码; 还要做平均才是交叉熵'''
    return - torch.log(y_hat[range(len(y_hat)), y])

# 根据预测结果（n*q->1*n）结合真的标记y(1*n)来判断判断正确的样例数。
def accuracy(y_hat, y):  #@save
    """返回 [预测正确的数量] , return/len(y)得到精确度。"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        # argmax: 对 y_hat 在第 1 轴（即每一行）上取最大值的索引，这些索引代表预测的类别标签。y_hat: n*q(q为输出分类数)
        y_hat = y_hat.argmax(axis=1)
    # 比较的结果是一个布尔数组，表示每个预测是否正确。
    cmp = (y_hat.type(y.dtype) == y)
    return float(cmp.type(y.dtype).sum())

# n维列表求和器
class Accumulator:  #@save
    """在n个变量上累加"""
    # self.data 是一个长度为 n 的列表，初始值为 [0.0, 0.0, ..., 0.0]
    # n，表示要累加的变量数量。
    def __init__(self, n):
        self.data = [0.0] * n

    # 接受可变数量的参数 *args。
    def add(self, *args):
        # 列表推导式，这里使用的是列表推导式来进行元素的逐个累加，创建新列表。
        # 格式：[expression for item in iterable]
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 计算总的预测分类精确度。
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    acc = Accumulator(2)  # 变量数 n=2
    with torch.no_grad():
        # X一个批次的训练数据：[batch_size, 1, 28, 28], y是分类标记[batch_size]
        for X, y in data_iter:
            # acc第一维是推测正确的样例数量，第二维是总样例数。
            acc.add(accuracy(net(X), y), y.numel())
    return acc[0] / metric[1]

def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """训练模型一个迭代周期（定义见第3章），返回平均损失和平均准确度"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数：清除之前的梯度，计算当前批次的梯度，更新模型参数。
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            l.sum().backward()
            # 使用定制的优化器和损失函数，这里的 X.shape[0] 表示批次大小。X:n*q
            updater(X.shape[0])
        # 损失总和，当前批次的准确预测数量，训练或测试样例总数量
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度(准确度)
    return metric[0] / metric[2], metric[1] / metric[2]

In [ ]:
class Animator:  #@save
    """在动画中绘制数据"""
    # xlabel,ylabel:x轴，y轴标签； 
    # legend:图例是在图表中标识不同数据系列的标签(多条线时用来表示区分不同线条含义)
    # xlim:包含两个值的元组，分别表示轴的最小值和最大值; 
    # xscale:'linear'：线性缩放（默认值），坐标轴上的间隔是等距的。
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        # axes：轴对象，是一个包含子图的二维数组（或者列表）
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        # 如果 nrows * ncols 等于 1，self.axes 的类型是单个对象，而不是列表。
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        '''向图表中添加多个数据点'''
        # 检查变量 y 是否具有 '__len__' 属性, 如果 y 是一个 [可迭代的对象]，则它应该具有 '__len__' 属性。
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        # 如果为空，即表示之前没有数据被添加过，那么分别初始化为 [具有 n 行、每行有 0 列的矩阵]。(二维数组)
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                # self.X[i] 是一个列表
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        # 重新显示和清除图形，以便在更新图表时刷新输出。
        display.display(self.fig)
        display.clear_output(wait=True)

In [11]:
import torch
# 真实的类别
y = torch.tensor([0, 2])
# 预测概率n*q（n个测试样例，q个分类）
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])

# 访问某个维度的大小
print(y_hat.shape,y_hat.shape[0])
# 选择了每个样本的真实类别的预测概率。y_hat[0, y[0]],y_hat[1, y[1]] (二维数组访问方式)
print(y_hat[[0, 1], y])
# 交叉熵的一部分
print(cross_entropy(y_hat, y))
# 初始化0列表
print([0.0]*4)

torch.Size([2, 3]) 2
tensor([0.1000, 0.5000])
tensor([2.3026, 0.6931])
[0.0, 0.0, 0.0, 0.0]
